In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from bs4 import BeautifulSoup

/tmp/ipykernel_26655/1023897703.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
service = Service()
options = ChromeOptions()


In [3]:
driver = webdriver.Chrome(service=service, options=options)



In [4]:
def pesquisa_processo(cpf = None, nome = None):
    if nome and not cpf:
        info = nome
        campo_pesquisa = driver.find_element(By.XPATH, '//*[@id="fPP:dnp:nomeParte"]')
    elif cpf and  not nome:
        info = cpf
        campo_pesquisa = driver.find_element(By.XPATH, '//*[@id="fPP:dpDec:documentoParte"]')
    
    campo_pesquisa.click()
    campo_pesquisa.send_keys(info)
    
    driver.find_element(By.XPATH, '//*[@id="fPP:searchProcessos"]').click()
    
def lista_processos():
    
    num_resultado = WebDriverWait(driver, 200).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="fPP:processosTable:tb"]/tr[1]'))
        )
    tabela = driver.find_element(By.XPATH, '//*[@id="fPP:processosGridPanel_body"]')
    
    html_tabela = tabela.get_attribute('outerHTML')
    soup = BeautifulSoup(html_tabela, 'html.parser')
    
    dados_tabela = []

    for linha in soup.find_all('tr'):
        dados_linha = []
        
        link_tag = linha.find('a', class_= 'btn btn-default btn-sm')
        if link_tag:
            link = link_tag.get('onclick').split("'")[3]  # extrai o link do atributo onclick
            link = 'https://pje1g.trf1.jus.br' + link
            
        dados_linha += [coluna.get_text(strip=True) for coluna in linha.find_all(['th', 'td'])]
        
        if 'Ver detalhes do processo' in dados_linha:
            indice = dados_linha.index('Ver detalhes do processo')
            dados_linha[indice] = link
        
        dados_tabela.append(dados_linha)
    

    df = pd.DataFrame(dados_tabela, columns=['link', 'processo', 'ultima_movimentacao'])
    df = df.iloc[2:]
    df.reset_index(drop=True, inplace=True)
    
    return df

def save_csv(df):
    df.to_csv('lista_processos', index = False)


In [5]:
url = 'https://pje1g.trf1.jus.br/consultapublica/ConsultaPublica/listView.seam'
driver.get(url)

nome = 'Roberto Barroso'
pesquisa_processo(nome=nome)
df = lista_processos()
save_csv(df)

In [6]:
df

,link,processo,ultima_movimentacao
0,https://pje1g.trf1.jus.br/consultapublica/Cons...,CUMPRIMENTO DE SENTENÇACumSen 0001955-72.1989....,Recebidos os Autos pela Contadoria (09/11/2023...
1,https://pje1g.trf1.jus.br/consultapublica/Cons...,CUMPRIMENTO DE SENTENÇACumSen 0011207-60.2007....,Juntada de petição intercorrente (07/12/2023 1...
2,https://pje1g.trf1.jus.br/consultapublica/Cons...,EMBARGOS À EXECUÇÃOEE 0014108-98.2007.4.01.350...,Processo Suspenso ou Sobrestado por Por decisã...
3,https://pje1g.trf1.jus.br/consultapublica/Cons...,EXECUÇÃO FISCALExFis 0020353-98.2007.4.01.3800...,Baixa Definitiva (26/08/2022 19:48:36)
4,https://pje1g.trf1.jus.br/consultapublica/Cons...,PROCEDIMENTO DO JUIZADO ESPECIAL CÍVELPJEC 001...,Conclusos para decisão (18/12/2023 09:51:18)
5,https://pje1g.trf1.jus.br/consultapublica/Cons...,EXECUÇÃO FISCALExFis 0000473-20.2012.4.01.4100...,Arquivado Provisoramente (29/11/2023 00:17:18)
6,https://pje1g.trf1.jus.br/consultapublica/Cons...,EXECUÇÃO FISCALExFis 0013671-11.2013.4.01.3900...,Arquivado Definitivamente (20/09/2023 21:02:55)
7,https://pje1g.trf1.jus.br/consultapublica/Cons...,CRIMES AMBIENTAISCriAmb 0018394-39.2013.4.01.3...,Juntada de manifestação (27/07/2023 10:01:23)
8,https://pje1g.trf1.jus.br/consultapublica/Cons...,EXECUÇÃO FISCALExFis 0017964-53.2014.4.01.3200...,Conclusos para julgamento (05/02/2024 15:20:12)
9,https://pje1g.trf1.jus.br/consultapublica/Cons...,AÇÃO CIVIL COLETIVAACC 0081910-78.2014.4.01.34...,None
